In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import requests  
import bs4 as bs 
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
train = pd.read_csv("/Users/utkarshmittal/Desktop/Education/Plaksha/Term 3/Project and Application data scienc/word2vec-nlp-tutorial/labeledTrainData.tsv", header=0,delimiter="\t", quoting=3)
test = pd.read_csv("/Users/utkarshmittal/Desktop/Education/Plaksha/Term 3/Project and Application data scienc/word2vec-nlp-tutorial/testData.tsv", header=0,delimiter="\t", quoting=3)

In [3]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
from nltk.corpus import stopwords
import re

def review_cleaner(review):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    #4. Tokenize into words (all lower case)
    review = review.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review = [w for w in review if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    review = ' '.join(review+emoticons)
    # add emoticons to the end

    return(review)

In [5]:
%%time

num_reviews = len(train['review'])

review_clean_original = []

for i in range(0,num_reviews):
    if( (i+1)%5000 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    review_clean_original.append(review_cleaner(train['review'][i]))

Done with 5000 reviews
Done with 10000 reviews
Done with 15000 reviews
Done with 20000 reviews
Done with 25000 reviews
CPU times: user 11.7 s, sys: 976 ms, total: 12.7 s
Wall time: 12.7 s


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    review_clean_original, train['sentiment'], random_state=0, test_size=.2)


vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [7]:
vectorizer.fit(X_train)


CountVectorizer(max_features=5000)

In [8]:
print(vectorizer.get_feature_names()[:10])


['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely']


In [9]:
train_bag = vectorizer.transform(X_train) 
test_bag = vectorizer.transform(X_test)

In [10]:
print(train_bag.toarray().shape) 
print(test_bag.toarray().shape)

(20000, 5000)
(5000, 5000)


In [11]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 50,n_jobs=-1)
forest.fit(train_bag, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=-1)

In [12]:
metrics.accuracy_score(y_train,forest.predict(train_bag))

1.0

In [13]:
metrics.accuracy_score(y_test,forest.predict(test_bag))

0.8398

In [14]:
forest.classes_

array([0, 1])

In [15]:
import pickle
pickle.dump(forest, open('/Users/utkarshmittal/Desktop/Education/Plaksha/Term 3/Project and Application data scienc/flask-ml-model-1/model.pkl','wb'))

In [16]:
pickle.dump(vectorizer, open('/Users/utkarshmittal/Desktop/Education/Plaksha/Term 3/Project and Application data scienc/flask-ml-model-1/vectorizer.pkl','wb'))